#### Task 1: Simple vector embedding generation

**Objective:**
Generate vector embeddings from text data.

**Task Description:**

- load huggingface embedding model (`model_name="sentence-transformers/all-mpnet-base-v2"`)
- embed simple text queries

How to select the right embedding model: [MTEB - Massive Text Embedding Benchmark](https://huggingface.co/blog/mteb)

**Useful links:**

- [Langchain Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)


In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

# ADD HERE YOUR CODE
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/workspaces/TeamMaxiKing/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/workspaces/TeamMaxiKing/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
text = "This is a test document."

# ADD HERE YOUR CODE
# Perform vector search
query_vector = embedding_model.embed_query(text=text)

print(f"Embedding vector length: {len(query_vector)}")
print(query_vector[:10])

Embedding vector length: 768
[-0.04895174503326416, -0.039861928671598434, -0.021562742069363594, 0.009908485226333141, -0.03810397908091545, 0.012684328481554985, 0.04349445924162865, 0.07183387875556946, 0.00974861066788435, -0.006987082771956921]


#### Task 2: Generate embedding vectors with custom dataset

**Objective:**
Load custom dataset, preprocess it and generate vector embeddings.

**Task Description:**

- load pdf document "AI_Book.pdf" via langchain document loader: `PyPDFLoader`
- use RecursiveCharacterTextSplitter to split documents into chunks
- generate embeddings for single documents

**RecursiveCharacterTextSplitter:**

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is `["\n\n", "\n", " ", ""]`. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

**Useful links:**

- [Langchain PyPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)
- [Langchain RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

pdf_doc = "./AI_Book.pdf"

# Create pdf data loader
# ADD HERE YOUR CODE
loader = PyPDFLoader(file_path=pdf_doc)

# Load and split documents in chunks
# ADD HERE YOUR CODE
pages = loader.load()
pages_chunked = RecursiveCharacterTextSplitter().split_documents(pages)

# Function to clean text by removing invalid unicode characters, including surrogate pairs
def clean_document_text(chunk):
    # Remove surrogate pairs
    text = chunk.page_content
    text = re.sub(r'[\ud800-\udfff]', '', text)
    # Optionally remove non-ASCII characters (depends on your use case)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return Document(page_content=text, metadata=chunk.metadata)

pages_chunked_cleaned = [clean_document_text(chunk) for chunk in pages_chunked]

print(pages_chunked_cleaned[0])

page_content='Aurlien GronHands-on Machine Learning with
Scikit-Learn, Keras, and
TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent SystemsSECOND EDITION
Boston Farnham Sebastopol Tokyo Beijing Boston Farnham Sebastopol Tokyo Beijing' metadata={'source': './AI_Book.pdf', 'page': 2}


In [4]:
print(pages_chunked_cleaned[1])

page_content='978-1-492-03264-9
[LSI]Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
by Aurlien Gron
Copyright  2019 Aurlien Gron. All rights reserved.
Printed in the United States of America.
Published by OReilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.
OReilly books may be purchased for educational, business, or sales promotional use. Online editions are
also available for most titles ( http://oreilly.com ). For more information, contact our corporate/institutional
sales department: 800-998-9938 or corporate@oreilly.com .
Editor:  Nicole Tache
Interior Designer:  David FutatoCover Designer:  Karen Montgomery
Illustrator:  Rebecca Demarest
June 2019:  Second Edition
Revision History for the Early Release
2018-11-05: First Release
2019-01-24: Second Release
2019-03-07: Third Release
2019-03-29: Fourth Release
2019-04-22: Fifth Release
See http://oreilly.com/catalog/errata.csp?isbn=9781492032649  for release details.
The OReilly logo is a reg

In [5]:
print(f"Number of text chunks: {len(pages_chunked_cleaned)}")

Number of text chunks: 507


#### Task 3: Store vector embeddings from pdf document to ChromaDB vector database.

**Objective:**

Store vector embeddings into ChromaDB to store knowledge.

**Task Description:**

- create chromadb client
- create chromadb collection
- create langchain chroma db client
- store text document chunks and vector embeddings to vector databases

**Useful links:**

- [Langchain How To](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#initialization-from-client)


In [6]:
from langchain_chroma import Chroma
import chromadb
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

# ADD HERE YOUR CODE
# Create a collection

collection_name = "AI_Book"

collection = client.get_or_create_collection(collection_name)

# ADD HERE YOUR CODE
# Create chromadb
vector_db_from_client = Chroma(
    client=client,
    collection_name=collection_name,
    embedding_function=embedding_model
)

In [7]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(pages_chunked_cleaned[:50]))]

# ADD HERE YOUR CODE
vector_db_from_client.add_documents(documents=pages_chunked_cleaned[:50], id=uuids)

['7bad263a-799d-4237-913c-7dc1cce5395c',
 'ab39e5e6-caff-4b5f-93c7-ce06ef6676e0',
 'ed186336-a30c-4ea6-bd04-6ec59eb21e5a',
 '7534869e-c1cc-475c-a4d3-f0526a88d15a',
 'd2809161-695f-4a7d-9c14-b550a78565a7',
 'bdbc2b98-e171-4f9a-b10f-fd24ef5433a4',
 'a658c07f-7e98-4326-8c05-52af4049190a',
 'd8f4a41c-b0f0-45e2-949e-683a74ef31a1',
 '6f7263b4-3df2-42b7-8715-a45f49ee4c8e',
 'bda205f1-3dec-499d-ba3d-06b328bb21a5',
 '07da4f71-cbe3-4e89-9c1c-47bdb51bae0d',
 '73626f70-d4c3-47de-92e2-1308dd18b81c',
 'e7fc4348-66d0-4e97-862b-d92ed52e076d',
 'c8395b70-dd1e-4264-8bb7-1dcade049608',
 'cc5f59f6-59e0-4846-9ba6-e75bec5a892c',
 'ca4cab84-2432-4c0c-8b4f-c23d6eeb2a83',
 '61e48c22-1785-48e6-9375-a4e03230069e',
 '3fd77d92-fc93-4371-adf6-c0c0372b5c08',
 'c6acbb20-901d-405e-a14a-8ed8d6fa6e53',
 'ed3ba7cf-2d88-4003-a183-1d24bd2aa780',
 '4f5811af-c154-4385-a158-cdd0a09e8078',
 '6af598e7-8db4-4806-85cb-de9ea74d3ec2',
 '48b91962-25b5-4fe2-9cd3-e30d234817c7',
 'e789e041-a58c-4be8-8645-ef7e96df43d2',
 'ab29a4c0-68e4-

In [8]:
client.count_collections()

3

In [9]:
# client.delete_collection("ai_model_book")

#### Task 4: Access ChromaDB and perform vector search

**Objective:**

Use query to perform vector search against ChromaDB vector database

**Task Description:**

- define query
- run vector search
- print k=3 most similar documents

**Useful links:**

- [Langchain Query ChromaDB](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#query-directly)


In [11]:
search_query = "Types of Machine Learning Systems"

results = vector_db_from_client.similarity_search(
    search_query,
    k=3
)

for res in results:
    print(res.page_content)
    print(res.metadata)
    print("\n----------------\n")

Types of Machine Learning Systems
There are so many different types of Machine Learning systems that it is useful to
classify them in broad categories based on:
Whether or not they are trained with human supervision (supervised, unsuper
vised, semisupervised, and Reinforcement Learning)
Whether or not they can learn incrementally on the fly (online versus batch
learning)
Whether they work by simply comparing new data points to known data points,
or instead detect patterns in the training data and build a predictive model, much
like scientists do (instance-based versus model-based learning)
These criteria are not exclusive; you can combine them in any way you like. For
example, a state-of-the-art spam filter may learn on the fly using a deep neural net
work model trained using examples of spam and ham; this makes it an online, model-
based, supervised learning system.
Lets look at each of these criteria a bit more closely.
Supervised/Unsupervised Learning
Machine Learning systems can be